# Problem Statement: **CNN-Based Fashion Classification by AtliQ**

### AtliQ is launching a state-of-the-art "**Fashion Classification System**" to automatically identify clothing items and accessories from images. The goal is to build a Convolutional Neural Network (CNN) model that can classify images from the **FashionMNIST** dataset into one of its 10 categories. You'll implement an end-to-end solution using PyTorch, covering dataset preparation, CNN construction, training, evaluation, and visualization.


Imports and CUDA

In [ ]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torchvision
from torch.utils.data import DataLoader, TensorDataset, random_split
from torchvision import datasets, transforms
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import pandas as pd

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")




---



**Step1**: Dataset Overview

Load the **FashionMNIST dataset** and extract a smaller subset for training and testing.

Details:
* Fashion-MNIST is a dataset of Zalando's article images—consisting of a training set of 60,000 examples and a test set of 10,000 examples. Each example is a 28x28 grayscale image, associated with a label from 10 classes.

* Use **10,000 images for training** and **3,000 images for testing** by randomly sampling from the dataset.
* Apply transformations such as random horizontal flips and random cropping for data augmentation during training.
* Normalize the pixel values to have a mean of 0 and standard deviation of 1 for faster convergence.
* Use `batch size = 64` for trainloader and testloader


Use torch.utils.data.random_split to partition the data into the desired subset sizes.

In [ ]:
# Define transformations
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(28, padding=4),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])  # Single-channel mean and std
])

# Load FashionMNIST dataset
dataset = # Code Here

test_dataset = # Code Here

# Subset the dataset (use random_split)
train_subset_size, test_subset_size = # Code Here
train_subset, _ = # Code Here
test_subset, _ = # Code Here

# Create data loaders
train_loader = # Code Here
test_loader = # Code Here




---



**Step2** :Visualizing the Dataset

Let's visualize some sample images with their class labels.

Details:
* Create a 4x4 grid of 16 images from the training dataset.
* Include the class labels (e.g., 0, 1, 2, 3 etc.) under each image

Label-Image Description:
* 0: T-shirt/top
* 1: Trouser
* 2: Pullover
* 3: Dress
* 4: Coat
* 5: Sandal
* 6: Shirt
* 7: Sneaker
* 8: Bag
* 9: Ankle boot

In [ ]:
classes = dataset.classes
# Retrieve a batch of data from the DataLoader
dataiter = iter(train_loader)
images, labels = next(dataiter)

print(type(images))
print(images.shape)
print(labels.shape)




---



In [ ]:
# Visualize a 4x4 grid of 16 images from the DataLoader
images, labels = next(iter(train_loader))

fig, axes = plt.subplots(4, 4, figsize=(8, 8))
for i, ax in enumerate(axes.flat):
    ax.imshow(images[i][0].numpy(), cmap='gray')  # Display image
    ax.set_title(f"Label: {labels[i].item()}")
    ax.axis('off')

plt.tight_layout()
plt.show()




---



**Step3**: Constructing the CNN Model

Let's build a convolutional neural network to classify FashionMNIST images.

**Details:**

* Use two convolutional layers followed by pooling, activation functions (ReLU), and fully connected layers.

**CNN components:**
* Convolutions: To extract spatial features from images.
* Pooling: To reduce dimensionality while retaining important information.
* Padding: To maintain image dimensions after convolutions.
* Strides: To control the movement of the kernel over the image.

**Architecture:**

* Conv1: Input channels = 1, Output channels = 32, Kernel size = 3x3, Activation = ReLU.
* Conv2: Input channels = 32, Output channels = 64, Kernel size = 3x3, Activation = ReLU.
* MaxPooling: Kernel size = 2x2, Stride = 2.

Fully Connected Layers:
* Input = 64*7*7, Output = 128, Activation = ReLU, Dropout = 0.5.
* Output = 10 (number of classes).

In [ ]:
class FashionMNISTCNN(nn.Module):
    def __init__(self):
        super(FashionMNISTCNN, self).__init__()
        self.features = nn.Sequential(
            # Conv block 1

            # ReLU

            # MaxPool

            # Conv block 2

            # ReLU

            # MaxPool
        )

        # Flatten Layer

        # Fully Connected Layers (Classifier)
        self.classifier = nn.Sequential(
            nn.Linear(64 * 7 * 7, 128),  # 64 channels, 7x7 feature map size after pooling
            nn.ReLU(),
            nn.Dropout(0.5),  # Dropout for regularization
            nn.Linear(128, 10),  # 10 output classes (for FashionMNIST)
            nn.LogSoftmax(dim=1)  # LogSoftmax for probabilistic output
        )

    def forward(self, x):
        x = self.features(x)
        x = self.flatten(x)
        x = self.classifier(x)
        return x

# Initialize the Model
model = # Code Here

# Print the model architecture




---



**Step 5**: Defining Loss Function and Optimizer

Now we'll establish the optimization and evaluation criteria for training.

**Details:**

* Use Cross-Entropy Loss for multi-class classification.
* Choose Adam optimizer for faster convergence and adaptive learning rates.

Mention the learning rate and other hyperparameters:
* Learning rate = 0.001
* betas for Adam = (0.9, 0.99)

In [ ]:
loss_fn = # Code Here
optimizer = # Code Here




---



**Step6**: Training the CNN Model

We will train the model on the training subset for multiple epochs.

**Details:**

* Train for **10 epochs**, where each epoch involves multiple iterations over the training subset.

Quick Revision:

* Forward pass: The model predicts outputs based on input images.
* Loss calculation: Compare predictions with true labels using the loss function.
* Backward pass: Calculate gradients using backpropagation.
* Optimizer step: Update model weights using the optimizer.
* Track the loss for each epoch to monitor convergence.

In [ ]:
num_epochs =
train_losses = []

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = # Code Here
        loss = # Code Here
        loss.backward()
        optimizer.step()
        running_loss # Code Here

    epoch_loss = running_loss / len(train_loader)
    train_losses.append(epoch_loss)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")




---



**Step7**: Evaluating the Model

Now we'll measure the performance of the trained CNN on the test subset.

**Details:**
* Use the test subset to evaluate accuracy.
* Explain the evaluation loop:
* Set the model to evaluation mode (disables dropout and other training-only features).
* Perform a forward pass for the test images.
* Compare predictions with true labels to calculate accuracy.


In [ ]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = # Code Here
        _, predicted = # Code Here
        total += # Code Here
        correct += # Code Here

accuracy = 100 * correct / total
print(f"Accuracy on test data: {accuracy:.2f}%")




---



**Step8**: Visualizing Predictions

**Details:**
* Randomly select 25 test images.
* Create a 5x5 grid showing the images, their predicted labels, and their true labels.



In [ ]:
# Define class labels for FashionMNIST
classes = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle Boot']

# Visualize 9 images with predictions and true labels
plt.figure(figsize=(15, 15))
for i in range(25):
    # Code here
    plt.imshow(images[i].cpu().squeeze(), cmap='gray')  # Unnormalize for grayscale
    plt.title(f"Pred: {classes[predicted[i]]}, True: {classes[labels[i]]}")
    plt.axis('off')
plt.tight_layout()
plt.show()




---

